In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SQL1").getOrCreate()

core= spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
core

1

In [3]:
spark

In [5]:
path = '../../data/youtubevideos.csv'

youtube = spark.read.csv(path,inferSchema=True,header=True)
youtube.limit(5).show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [8]:
youtube.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [9]:
youtube.columns

['video_id',
 'trending_date',
 'title',
 'channel_title',
 'category_id',
 'publish_time',
 'tags',
 'views',
 'likes',
 'dislikes',
 'comment_count',
 'thumbnail_link',
 'comments_disabled',
 'ratings_disabled',
 'video_error_or_removed',
 'description']

In [10]:
from pyspark.sql.types import * 
from pyspark.sql.functions import * 

youtube = youtube.withColumn("views",youtube.views.cast(IntegerType())).withColumn('likes',youtube.likes.cast(IntegerType()))\
          .withColumn("dislikes",youtube.dislikes.cast(IntegerType())).withColumn('comment_count',youtube.comment_count.cast(IntegerType()))\
        .withColumn("comments_disabled",youtube.comments_disabled.cast(BooleanType())).withColumn("ratings_disabled",youtube.ratings_disabled.cast(BooleanType()))\
        .withColumn("video_error_or_removed",youtube.video_error_or_removed.cast(BooleanType()))

youtube.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- video_error_or_removed: boolean (nullable = true)
 |-- description: string (nullable = true)



In [12]:
youtube.createOrReplaceTempView("youtube")

spark.sql("SELECT * FROM YOUTUBE").toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374.0,57527.0,2966.0,15954.0,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"""last week tonight trump presidency""|""last wee...",2418783.0,97185.0,6146.0,12703.0,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434.0,146033.0,5339.0,8181.0,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"""rhett and link""|""gmm""|""good mythical morning""...",343168.0,10172.0,666.0,2146.0,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",2095731.0,132235.0,1989.0,17518.0,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48132,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,2018-05-18T13:00:04.000Z,"""aarons animals""|""aarons""|""animals""|""cat""|""cat...",1685609.0,38160.0,1385.0,2657.0,https://i.ytimg.com/vi/BZt0qjTWNhw/default.jpg,False,False,False,The Cat Who Caught the Laser - Aaron's Animals
48133,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,2018-05-18T01:00:06.000Z,[none],1064798.0,60008.0,382.0,3936.0,https://i.ytimg.com/vi/1h7KV2sjUWY/default.jpg,False,False,False,None
48134,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,2018-05-18T17:34:22.000Z,"""I gave safiya nygaard a perfect hair makeover...",1066451.0,48068.0,1032.0,3992.0,https://i.ytimg.com/vi/D6Oy4LfoqsU/default.jpg,False,False,False,I had so much fun transforming Safiyas hair in...
48135,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,2018-05-17T17:00:04.000Z,"""Black Panther""|""HISHE""|""Marvel""|""Infinity War...",5660813.0,192957.0,2846.0,13088.0,https://i.ytimg.com/vi/oV0zkMe1K8s/default.jpg,False,False,False,How Black Panther Should Have EndedWatch More ...


In [15]:
#category에 따른 좋아요 수 평균

spark.sql("SELECT category_id,avg(likes) FROM YOUTUBE GROUP BY category_id limit 5").toPandas()




,category_id,avg(likes)
0,15,21055.110870
1,artist,NaN
2,informed reporting of everything from wars an...,NaN
3,29,259923.614035
4,dining destinations,NaN


In [43]:
spark.sql("""SELECT category_id, sum(likes) as total_likes 
            FROM YOUTUBE 
            GROUP BY category_id 
            HAVING total_likes is not null 
            ORDER BY total_likes desc""" ).toPandas()

#where 문 통해서, category_id가 숫자형일때? -> 가능.
#having문 통해서 sum(likes)가 null이 아닐 때. -> 가능.




,category_id,total_likes
0,10,1416838584
1,24,530516491
2,23,216346746
3,22,186615999
4,1,165997476
5,26,162880075
6,17,98621211
7,28,82532638
8,20,69038284
9,27,49257772


In [64]:
#정규표현식 이용해서 분류! 
spark.sql("""SELECT *
            FROM (SELECT category_id,likes,dislikes from YOUTUBE where category_id REGEXP '([re0-9]*)')

            """ ).limit(3).toPandas()

,category_id,likes,dislikes
0,22,57527,2966
1,24,97185,6146
2,23,146033,5339


In [70]:
#게시물당 달린 태그의 갯수를 쿼리해보자 

spark.sql("""SELECT title,channel_title,tags, (length(tags) - length(replace(tags,'|',""))+1) as num_tags
            FROM YOUTUBE
            """ 
         ).toPandas()

,title,channel_title,tags,num_tags
0,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,SHANtell martin,1.0
1,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,"""last week tonight trump presidency""|""last wee...",4.0
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",23.0
3,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,"""rhett and link""|""gmm""|""good mythical morning""...",27.0
4,I Dare You: GOING BALD!?,nigahiga,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",14.0
...,...,...,...,...
48132,The Cat Who Caught the Laser,AaronsAnimals,"""aarons animals""|""aarons""|""animals""|""cat""|""cat...",14.0
48133,True Facts : Ant Mutualism,zefrank1,[none],1.0
48134,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,"""I gave safiya nygaard a perfect hair makeover...",24.0
48135,How Black Panther Should Have Ended,How It Should Have Ended,"""Black Panther""|""HISHE""|""Marvel""|""Infinity War...",22.0


In [80]:
#채널당 게시물 숫자?

spark.sql("""SELECT CHANNEL_TITLE,COUNT(3) as count FROM YOUTUBE WHERE CHANNEL_TITLE is not null GROUP BY CHANNEL_TITLE ORDER BY 2 DESC 
             """).toPandas()

,CHANNEL_TITLE,count
0,ESPN,203
1,The Tonight Show Starring Jimmy Fallon,197
2,TheEllenShow,193
3,Netflix,193
4,Vox,193
...,...,...
2270,Wendy's,1
2271,Steve Kovach,1
2272,Katy Perry,1
2273,Absolut,1
